In [4]:
from matplotlib import pyplot
import pandas as pd
import numpy as np

## **Parte 1: Ecuación del calor**

La función debe devolver una matriz cuya entrada en la posición (i, j) represente la aproximación
numérica de la solución en el instante de tiempo t = i · ∆t y en la posición espacial x = j · ∆x.

### **Método Explicíto**

In [5]:
def metodo_explicito_calor(alpha, f, dx, dt, T):
    # Número de puntos espaciales (incluyendo fronteras)
    N = int(1/dx) + 1
    # Número de pasos temporales
    M = int(T/dt) + 1
    # Inicializar matriz solución
    U = np.zeros((M, N))
    # Vector de posiciones espaciales
    x = np.linspace(0, 1, N)
    # Aplicar condición inicial
    U[0, :] = f(x)
    # Condiciones de frontera (Dirichlet homogéneas)
    U[:, 0] = 0   # u(0,t) = 0
    U[:, -1] = 0  # u(1,t) = 0
    
    # Parámetro de estabilidad
    r = alpha * dt / (dx**2)
    
    # Verificar condición de estabilidad (r <= 0.5 para estabilidad)
    if r > 0.5:
        print(f"¡Advertencia! r = {r:.4f} > 0.5. El esquema puede ser inestable.")
        print(f"Para estabilidad, dt debería ser <= {0.5 * dx**2 / alpha:.6f}")
    
    for n in range(M-1):  # Para cada paso temporal
        for j in range(1, N-1):  # Para puntos interiores
            U[n+1, j] = U[n, j] + r * (U[n, j+1] - 2*U[n, j] + U[n, j-1])
    
    return U, x

print("Función del método explícito para la ecuación del calor creada exitosamente!")
print(f"Parámetro r de estabilidad debe ser <= 0.5 para estabilidad numérica")

Función del método explícito para la ecuación del calor creada exitosamente!
Parámetro r de estabilidad debe ser <= 0.5 para estabilidad numérica


### **Método Implicíto**

In [ ]:
#COMPLETAR

#### Evaluación de las funciones con los parámetros: α = 1, ∆t = 0,001, ∆x = 0,05, T = 0,1

In [7]:
# Parámetros dados
alpha = 1.0      # Coeficiente de difusión
dt = 0.001       # Paso temporal
dx = 0.05        # Paso espacial  
T = 0.1          # Tiempo final

# Definir condición inicial (corregida)
def f(x):
    # Función triangular: 1 - |2x - 1| para x en [0,1]
    return 1 - np.abs(2 * x - 1)

# Ejecutar método explícito
U_explicito, x_grid = metodo_explicito_calor(alpha, f, dx, dt, T)

print(f"Dimensiones de la matriz solución: {U_explicito.shape}")
print(f"Número de puntos espaciales: {len(x_grid)}")
print(f"Número de pasos temporales: {U_explicito.shape[0]}")

# Calcular parámetro r para verificar estabilidad
r = alpha * dt / (dx**2)
print(f"\nParámetro de estabilidad r = {r:.4f}")
print(f"Condición de estabilidad: r <= 0.5 → {r <= 0.5}")

# Mostrar algunos valores de la solución
print(f"\nCondición inicial (t=0):")
print(f"u(0.25, 0) = {U_explicito[0, int(0.25/dx)]:.4f}")
print(f"u(0.50, 0) = {U_explicito[0, int(0.50/dx)]:.4f}")
print(f"u(0.75, 0) = {U_explicito[0, int(0.75/dx)]:.4f}")

print(f"\nSolución en t = T = {T}:")
print(f"u(0.25, {T}) = {U_explicito[-1, int(0.25/dx)]:.4f}")
print(f"u(0.50, {T}) = {U_explicito[-1, int(0.50/dx)]:.4f}")
print(f"u(0.75, {T}) = {U_explicito[-1, int(0.75/dx)]:.4f}")

Dimensiones de la matriz solución: (101, 21)
Número de puntos espaciales: 21
Número de pasos temporales: 101

Parámetro de estabilidad r = 0.4000
Condición de estabilidad: r <= 0.5 → True

Condición inicial (t=0):
u(0.25, 0) = 0.5000
u(0.50, 0) = 1.0000
u(0.75, 0) = 0.5000

Solución en t = T = 0.1:
u(0.25, 0.1) = 0.2134
u(0.50, 0.1) = 0.3019
u(0.75, 0.1) = 0.2134


GIF - Con los valores devueltos en el caso anterior realizar un GIF de como al variar el instante de tiempo evolucionan los valores en cada lugar espacial. Incluir un pequeño análisis sobre el comportamiento de los métodos según los diferentes parámetros.


In [ ]:
# Visualización de la evolución temporal
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Crear gráfico de evolución temporal
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Perfiles de temperatura en diferentes tiempos
times_to_plot = [0, int(0.25*len(U_explicito)), int(0.5*len(U_explicito)), 
                 int(0.75*len(U_explicito)), -1]
time_labels = ['t=0', 't=T/4', 't=T/2', 't=3T/4', f't=T={T}']

ax1.set_title('Evolución de la temperatura en diferentes tiempos')
ax1.set_xlabel('Posición x')
ax1.set_ylabel('Temperatura u(x,t)')
ax1.grid(True, alpha=0.3)

for i, time_idx in enumerate(times_to_plot):
    ax1.plot(x_grid, U_explicito[time_idx, :], 
             label=time_labels[i], marker='o', markersize=4)

ax1.legend()

# Gráfico 2: Mapa de calor (heatmap)
im = ax2.imshow(U_explicito, aspect='auto', origin='lower', 
                extent=[0, 1, 0, T], cmap='hot')
ax2.set_title('Mapa de calor: Evolución espacio-temporal')
ax2.set_xlabel('Posición x')
ax2.set_ylabel('Tiempo t')
plt.colorbar(im, ax=ax2, label='Temperatura u(x,t)')

plt.tight_layout()
plt.show()

# Análisis de conservación de energía
def calcular_energia(U, dx):
    """Calcula la energía total en cada instante de tiempo"""
    # Energía = integral de u²(x,t) dx ≈ suma de u²(x,t) * dx
    energia = []
    for n in range(U.shape[0]):
        energia.append(np.sum(U[n, :]**2) * dx)
    return np.array(energia)

energia = calcular_energia(U_explicito, dx)
tiempo = np.linspace(0, T, len(energia))

plt.figure(figsize=(10, 6))
plt.plot(tiempo, energia, 'b-', linewidth=2)
plt.title('Evolución de la energía total del sistema')
plt.xlabel('Tiempo t')
plt.ylabel('Energía total')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Energía inicial: {energia[0]:.6f}")
print(f"Energía final: {energia[-1]:.6f}")
print(f"Pérdida de energía: {((energia[0] - energia[-1])/energia[0]*100):.2f}%")

## **Parte 2: Ecuación de transporte**


### **Método Explicíto**

In [ ]:
#COMPLETAR


### **Método Implicíto**

In [ ]:
#COMPLETAR

#### Evaluación de las funciones con los parámetros: α = 1, ∆t = 0,001, ∆x = 0,05, T = 0,1

In [ ]:
#COMPLETAR
# Definir condición inicial
def f(x):
    return np.sin(np.pi * x)


GIF - Con los valores devueltos en el caso anterior realizar un GIF de como al variar el instante de tiempo evolucionan los valores en cada lugar espacial. Incluir un pequeño análisis sobre el comportamiento de los métodos según los diferentes parámetros.


In [ ]:
#COMPLETAR